In [1]:
import numpy as np

class Physical():

    def __init__(self, value, unit=""):
        self.value = value # store the numerical value as a plain numpy array
        self.unit = unit

    def __repr__(self):
        return f"<Physical:({self.value}, {self.unit})"

    def __str__(self):
        return f"{self.value} {self.unit}"

weights = Physical(np.array([55.6, 45.7, 80.3]), "kilogram")
print(weights)

[55.6 45.7 80.3] kilogram


In [2]:
class Physical():
    
    def __init__(self, value, unit=""):
        self.value = value
        self.unit = unit
    
    def __repr__(self):
        return f"<Physical:({self.value}, {self.unit})"
    
    def __str__(self):
        return f"{self.value} {self.unit}"
    
    def __add__(self, other):
        if self.unit == other.unit:
            return Physical(self.value + other.value, self.unit)
        else:
            raise ValueError("Physical objects must have same unit to be added.")
    
    def __sub__(self, other):
        if self.unit == other.unit:
            return Physical(self.value - other.value, self.unit)
        else:
            raise ValueError("Physical objects must have same unit to be subtracted.")

    def __mul__(self, other):
        return Physical(self.value * other.value, f"{self.unit}*{other.unit}")
    
    def __truediv__(self, other):
        return Physical(self.value / other.value, f"{self.unit}/{other.unit}")
        
    def __pow__(self, powfac):
        return Physical(self.value**powfac, f"{self.unit}^{powfac}")
        
weights = Physical(np.array([55.6, 45.7, 80.3]), "kilogram")
heights = Physical(np.array([1.64, 1.85, 1.77]), "meter")
print(weights)
print(heights)
print(heights + heights)
# print(height + weights) # raises ValueError
print(heights**2)

[55.6 45.7 80.3] kilogram
[1.64 1.85 1.77] meter
[3.28 3.7  3.54] meter
[2.6896 3.4225 3.1329] meter^2


In [3]:
weights = Physical(np.array([55.6, 45.7, 80.3]), "kilogram")
heights = Physical(np.array([1.64, 1.85, 1.77]), "meter")
bmi = weights/heights**2
print(bmi)

[20.67221892 13.35281227 25.63120432] kilogram/meter^2


In [4]:
# FIRST
HANDLED_FUNCTIONS = {}

class Physical():
    
    def __init__(self, value, unit=""):
        self._value = value
        self._unit = unit
    
    def __repr__(self):
        return f"<Physical:({self._value}, {self._unit})"
    
    def __str__(self):
        return f"{self._value} {self._unit}"
    
    def __add__(self, other):
        if self._unit == other._unit:
            return Physical(self._value + other._value, self._unit)
        else:
            raise ValueError("Physical objects must have same unit to be added.")
    
    def __sub__(self, other):
        if self._unit == other._unit:
            return Physical(self._value - other._value, self._unit)
        else:
            raise ValueError("Physical objects must have same unit to be subtracted.")

    def __mul__(self, other):
        return Physical(self._value * other._value, f"{self._unit}*{other._unit}")
    
    def __truediv__(self, other):
        return Physical(self._value / other._value, f"{self._unit}/{other._unit}")
    
    def __pow__(self, powfac):
        return Physical(self._value**powfac, f"{self._unit}^{powfac}")
        
    def __array_function__(self, func, types, args, kwargs):
        if func not in HANDLED_FUNCTIONS:
            return NotImplemented
        # Note: this allows subclasses that don't override
        # __array_function__ to handle Physical objects
        if not all(issubclass(t, Physical) for t in types):
            return NotImplemented
        return HANDLED_FUNCTIONS[func](*args, **kwargs)
    
# THIRD
def implements(numpy_function):
    """Register an __array_function__ implementation for Physical objects."""
    def decorator(func):
        HANDLED_FUNCTIONS[numpy_function] = func
        return func
    return decorator
    
# FOURTH
@implements(np.mean)
def np_mean_for_physical(x, *args, **kwargs):
    # first compute the numerical value, with no notion of unit
    mean_value = np.mean(x._value, *args, **kwargs)
    # construct a Physical instance with the result, using the same unit
    return Physical(mean_value, x._unit)
 
weights = Physical(np.array([55.6, 45.7, 80.3]), "kilogram")
heights = Physical(np.array([1.64, 1.85, 1.77]), "meter")
bmi = weights/heights**2

print(np.mean(bmi)) # 19.885411834844252 kilogram/meter^2

19.885411834844252 kilogram/meter^2


## Full Code

In [5]:
import numpy as np

HANDLED_FUNCTIONS = {}

class Physical():
    
    def __init__(self, value, unit=""):
        self._value = value
        self._unit = unit
    
    def __repr__(self):
        return f"<Physical:({self._value}, {self._unit})"
    
    def __str__(self):
        return f"{self._value} {self._unit}"
    
    def __add__(self, other):
        if self._unit == other._unit:
            return Physical(self._value + other._value, self._unit)
        else:
            raise ValueError("Physical objects must have same unit to be added.")
    
    def __sub__(self, other):
        if self._unit == other._unit:
            return Physical(self._value - other._value, self._unit)
        else:
            raise ValueError("Physical objects must have same unit to be subtracted.")

    def __mul__(self, other):
        return Physical(self._value * other._value, f"{self._unit}*{other._unit}")
    
    def __truediv__(self, other):
        return Physical(self._value / other._value, f"{self._unit}/{other._unit}")
    
    def __pow__(self, powfac):
        return Physical(self._value**powfac, f"{self._unit}^{powfac}")
        
    def __array_function__(self, func, types, args, kwargs):
        if func not in HANDLED_FUNCTIONS:
            return NotImplemented
        # Note: this allows subclasses that don't override
        # __array_function__ to handle Physical objects
        if not all(issubclass(t, Physical) for t in types):
            return NotImplemented
        return HANDLED_FUNCTIONS[func](*args, **kwargs)
    
    
def implements(numpy_function):
    """Register an __array_function__ implementation for Physical objects."""
    def decorator(func):
        HANDLED_FUNCTIONS[numpy_function] = func
        return func
    return decorator
    
    
@implements(np.mean)
def np_mean_for_physical(x, *args, **kwargs):
    # first compute the numerical value, with no notion of unit
    mean_value = np.mean(x._value, *args, **kwargs)
    # construct a Physical instance with the result, using the same unit
    return Physical(mean_value, x._unit)
 
    
weights = Physical(np.array([55.6, 45.7, 80.3]), "kilogram")
heights = Physical(np.array([1.64, 1.85, 1.77]), "meter")
print(weights)
print(heights)
print(heights + heights)
print(heights**2)
ratio = weights/heights
print(ratio)
bmi = weights/heights**2
print(bmi)
print(np.mean(bmi))

[55.6 45.7 80.3] kilogram
[1.64 1.85 1.77] meter
[3.28 3.7  3.54] meter
[2.6896 3.4225 3.1329] meter^2
[33.90243902 24.7027027  45.36723164] kilogram/meter
[20.67221892 13.35281227 25.63120432] kilogram/meter^2
19.885411834844252 kilogram/meter^2
